In [6]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build

In [3]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [21]:
###### Videos data

# Channels Ids (content = "https://www.youtube.com/channel/...) and titles:
channels = {
    "UCBa659QWEk1AI4Tg--mrJ2A": "Tom Scott",
    "UCXvA7wLxIJDa1SWYWl7e-oQ": "arewethereyet",
    "UCLXl1V6n82Dyg1VhVgSL0nw": "Aviões e Músicas com Lito Sousa",
    "UC32z4mtyiq02Ge-XWy78ibw": "Aero Por Trás da Aviação",
    "UC_qKzDKiOkCjkz0JKl1BufA": "Gary L. Francione",
    "UC-9b7aDP6ZN0coj9-xFnrtw": "Astrum",
    "UC04KsGq3npibMCE9Td3mVDg": "Cheddar",
    "UC1ZBQ-F-yktYD4m5AzM6pww": "Mustard",
    "UCHnyfMqiRRG1u-2MsSQLbXA": "Veritasium",
    "UC6uKrU_WqJ1R2HMTY3LIx5Q": "Everyday Astronaut",
    "UC6u6uY4VbvuNtU0BU7F9olw": "Studson Studio",
    "UCvJJ_dzjViJCoLf5uKUTwoA": "CNBC",
    "UC6n8I1UDTKP1IWjQMg6_TwA": "The B1M",
    "UCP5tjEmvPItGyLhmjdwP7Ww": "RealLifeLore",
    "UCsXVk37bltHxD1rDPwtNM8Q": "Kurzgesagt – In a Nutshell",
    "UCaRMivfyupj3ucUyJbZbCNg": "Anglo-Link",
    "UCAuUUnT6oDeKwE6v1NGQxug": "TED",
    "UCvn_XCl_mgQmt3sD753zdJA": "Rachel's English",
    "UCxzC4EngIsMrPmbm6Nxvb-A": "Scott Manley",
    "UCliwOcLaEfqkGJ7xq2ipq4w": "Reverse Engineering",
    "UCNqNkZ7kKfqimqHkgbWMNYA": "Launch Pad Astronomy",
    "UCZ9jWH_8tJ-Nmaj8dSQdEYA": "Stefan Milo",
    "UCbbQalJ4OaC0oQ0AqRaOJ9g": "Jay Foreman",
    "UC726J5A0LLFRxQ0SZqr2mYQ": "Curious Droid",
    "UCr22xikWUK2yUW4YxOKXclQ": "MathTheBeautiful",
    "UCzR-rom72PHN9Zg7RML9EbA": "PBS Eons",
    "UCRXRbi80k0_vcIfgpOSerTg": "Subject Zero Science",
    "UCYO_jab_esuFRV4b17AJtAw": "3Blue1Brown",
    "UCBbnbBWJtwsf0jLGUwX5Q3g": "Journey to the Microcosmos",
    "UC2LVhJH_9cT2XKp0VAfsKOQ": "The Tim Traveller",
    "UCQ3JxE-NOyZaJ3m3qIkZbhA": "O Pimentinha",
    "UCR1IuLEqb6UEA_zQ81kwXfg": "Real Engineering",
    "UCwK-svlNEASA7g_smu1_d8g": "OnlyConnect Fan",
    "UCS4iXNXsvlOHm7TuXLltreg": "chilli music",
    "UCR1-7g_y2YcYlh1W9y_1LUg": "Chaves Estranho",
    "UC6zbvGt_jJVgw7-MWkVUC5A": "Fala Vegan",
    "UCp3iXxis9n_E_GfbE-_ksFw": "VeganBlackMetalChef",
    "UCZ6JHFBaDUJ9wfo41HSII_w": "Fabio Chaves",
    "UCVRrGAcUc7cblUzOhI1KfFg": "Earthling Ed",
    "UCk5BcU1rOy6hepflk7_q_Pw": "Meteoro Brasil",
    "UCxcnsr1R5Ge_fbTu5ajt8DQ": "Bob Ross",
    "UCaWd5_7JhbQBe4dknZhsHJg": "WatchMojo.com",
    "UCtI0Hodo5o5dUb67FeUjDeA": "SpaceX",
    "UChDKyKQ59fYz3JO2fl0Z6sg": "TODAY",
    "UCi5iiEyLwSLvlqnMi02u5gQ": "Ask a Mortician",
    "UCZdJE8KpuFm6NRafHTEIC-g": "Tempero Drag",
    "UCpQLC-evmUAon9BBpcW4kYg": "Steve Cutts",
    "UCcMDMoNu66_1Hwi5-MeiQgw": "Hydraulic Press Channel",
    "UC9Ntx-EF3LzKY1nQ5rTUP2g": "cyriak",
    "UCBlXovStrlQkVA2xJEROUNg": "Marcel Vos"
}

# Loop in the channels:
df_videos = pd.DataFrame(
    {
        "channel_title": pd.Series([], dtype = "str"),
        "channel_id": pd.Series([], dtype = "str"),
        "video_title": pd.Series([], dtype = "str"),
        "video_id": pd.Series([], dtype = "str"),
        "video_upload_date": pd.Series([], dtype = "str"),
        "views": pd.Series([], dtype = "int"),
        "likes": pd.Series([], dtype = "int"),
        "dislikes": pd.Series([], dtype = "int"),
        "comments": pd.Series([], dtype = "int")
    }
)
for chnl_ind, chnl_key in enumerate(channels):
    # Choose the channel:
    channel_id = chnl_key
    channel_title = channels[channel_id]

    ### Extract the data from all the videos of the channel

    # Content details:
    content = yt.channels().list(
        id = channel_id,
        part = "contentDetails"
    ).execute()

    # Upload Id:
    upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # All the channel's videos:
    all_videos = []
    next_pg_token = None
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    while True:
        res = yt.playlistItems().list(
            playlistId = upload_id,
            maxResults = 50,
            part = "snippet",
            pageToken = next_pg_token
        ).execute()
        all_videos += res["items"]
        next_pg_token = res.get("nextPageToken")
        if next_pg_token is None:
            break

    # Videos statistics:
    videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
    stats = []
    for i in range(0, len(videos_ids), 40):
        res = (yt).videos().list(
            id = ",".join(videos_ids[i:i + 40]),
            part = "statistics"
        ).execute()
        stats += res["items"]

    # Put it in a dataframe:
    video_id, views, likes, dislikes, comments = [], [], [], [], []
    for i in range(len(stats)):
        video_id += [stats[i]["id"]]
        stats_i = stats[i]["statistics"]
        views += [stats_i[k] if "viewCount" in stats_i.keys() else np.nan for k in ["viewCount"]]
        likes += [stats_i[k] if "likeCount" in stats_i.keys() else np.nan for k in ["likeCount"]]
        dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else np.nan for k in ["dislikeCount"]]
        comments += [stats_i[k] if "commentCount" in stats_i.keys() else np.nan for k in ["commentCount"]]
    df_channel_videos = pd.DataFrame(
        {
            "channel_title": channel_title,
            "channel_id": channel_id,
            "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
            "video_id": video_id,
            "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
            "views": views,
            "likes": likes,
            "dislikes": dislikes,
            "comments": comments        
        }
    )
    df_videos = pd.concat([df_videos, df_channel_videos])

# Save the dataframe:
df_videos.to_csv("data/videos_data.csv",
                sep = ";",
                index = False)

In [154]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments
0,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's only float-through McDonalds,A6F96xSoLPg,2021-10-08T16:43:23Z,1992036,127268,896,4517
1,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,I thought the Schmid Peoplemover was impossible,A2g4u9F9i90,2021-09-30T16:02:45Z,2605088,142545,803,5613
2,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's most useful model railway,6TLcaJdsRr0,2021-09-24T14:00:54Z,1944306,107135,619,3176
3,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The public toll road with no speed limit,10Y-gWNJ2Sw,2021-09-17T08:11:31Z,1936707,128848,814,3952
4,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Three strange river crossings,KVXVNX4Crh4,2021-09-05T13:07:18Z,1468052,87205,428,1908
...,...,...,...,...,...,...,...,...,...
73,Marcel Vos,UC9Ntx-EF3LzKY1nQ5rTUP2g,granny-vision news update,kcVh-tXfwZw,2006-08-01T19:33:17Z,425161,5201,170,451
74,Marcel Vos,UC9Ntx-EF3LzKY1nQ5rTUP2g,Queenie,4SBXcybIo3o,2006-08-01T12:13:00Z,836764,8289,391,1063
75,Marcel Vos,UC9Ntx-EF3LzKY1nQ5rTUP2g,beastenders - deleted scene,IoYkY9mK5ys,2006-07-08T13:55:28Z,757181,7076,263,364
76,Marcel Vos,UC9Ntx-EF3LzKY1nQ5rTUP2g,Beastenders,j3S3Txsl2vs,2006-07-03T11:18:49Z,7757984,52685,5294,7891
